In [1]:
import pandas as pd


df = pd.read_excel("COHORT ANALYSIS.xlsx")

In [3]:

# Clean column names (remove symbols and replace spaces)
df.columns = df.columns.str.strip().str.replace(r'[^\w\s]', '', regex=True).str.replace(' ', '_')

# Corrected column names after cleaning
goal_col = 'Goal__spots_to_fill'
offers_col = 'Enrollment_offers_extended'

# Drop rows with missing values in relevant columns
df = df.dropna(subset=[goal_col, offers_col]).copy()

# Convert to numeric safely
df.loc[:, goal_col] = pd.to_numeric(df[goal_col], errors='coerce')
df.loc[:, offers_col] = pd.to_numeric(df[offers_col], errors='coerce')

# Calculate stats
cohorts_per_year = 6
avg_goal = df[goal_col].mean()
annual_acceptance_capacity = avg_goal * cohorts_per_year


# Output
print(f"✅ Avg. accepted applicants per cohort: {avg_goal:.0f}")
print(f"📆 Estimated annual acceptance capacity (6 cohorts): {annual_acceptance_capacity:.0f}")



✅ Avg. accepted applicants per cohort: 11
📆 Estimated annual acceptance capacity (6 cohorts): 66
🕓 Avg. waitlist per cohort: 4
📋 Estimated annual waitlist capacity: 25


In [9]:
# Focused columns
df2_filtered = df[['Applications_Submitted',
                  '_applicants_who_Completed_Application_Packet',
                  'Eligible_applicants',
                  'Enrolled__Week_2']].dropna()

# Calculate stage-to-stage conversion rates
df2_filtered['Submit_to_Complete_Rate'] = df2_filtered['_applicants_who_Completed_Application_Packet'] / df2_filtered['Applications_Submitted']
df2_filtered['Complete_to_Eligible_Rate'] = df2_filtered['Eligible_applicants'] / df2_filtered['_applicants_who_Completed_Application_Packet']
df2_filtered['Eligible_to_Enrolled_Rate'] = df2_filtered['Enrolled__Week_2'] / df2_filtered['Eligible_applicants']

# Calculate overall conversion rate from Applications Submitted to Enrolled
df2_filtered['Total_Conversion_Rate'] = df2_filtered['Enrolled__Week_2'] / df2_filtered['Applications_Submitted']

# Average total conversion rate
avg_conversion_rate = df2_filtered['Total_Conversion_Rate'].mean()

# Goal per cohort
target_enrolled = 11

# Calculate how many applicants you'd need (on average) to meet target
required_applicants = round(target_enrolled / avg_conversion_rate)

print(f"🎯 To consistently enroll {target_enrolled} students per cohort, you should aim for at least {required_applicants} applications submitted per cohort.")
print(f"📊 Based on an average conversion rate of {avg_conversion_rate:.2%} from applications to enrolled.")


🎯 To consistently enroll 11 students per cohort, you should aim for at least 52 applications submitted per cohort.
📊 Based on an average conversion rate of 21.33% from applications to enrolled.


In [13]:
# Clean column names (to remove spaces and special characters)
df.columns = df.columns.str.strip().str.replace(r'[^\w\s]', '', regex=True).str.replace(' ', '_')

# Define the relevant columns
offers_col = 'Enrollment_offers_extended'  # Total offers extended
enrolled_col = 'Enrolled__Week_2'  # Number of students enrolled by Week 2

# Drop rows with missing values in the relevant columns
df = df.dropna(subset=[offers_col, enrolled_col])

# Calculate No-Shows
df.loc[:, 'No_Shows'] = df[offers_col] - df[enrolled_col]

# Calculate No-Show Rate
df.loc[:, 'No_Show_Rate'] = df['No_Shows'] / df[offers_col]

# Calculate the average No-Show Rate across all cohorts
avg_no_show_rate = df['No_Show_Rate'].mean()

# Output the result
print(f"📉 Average No-Show Rate: {avg_no_show_rate:.2f} or {avg_no_show_rate*100:.0f}%")



📉 Average No-Show Rate: 0.23 or 23%


In [15]:

# Calculate the overall Acceptance Rate across all cohorts
total_graduates = df['Graduated'].sum()
total_offers_extended = df['Enrollment_offers_extended'].sum()

# Overall Acceptance Rate
overall_acceptance_rate = total_graduates / total_offers_extended

# Define target enrollment
target_enrollment = 11

# Calculate the generalized number of offers needed to meet the target enrollment
offers_needed = target_enrollment / overall_acceptance_rate

no_show_rate = avg_no_show_rate
# Calculate the number of waitlist offers based on no-show/withdrawal estimation
waitlist_offers_needed = (target_enrollment / (1 - no_show_rate)) - target_enrollment

# Calculate total offers needed (including waitlist offers)
total_offers_needed = offers_needed + waitlist_offers_needed

# Output the generalized offer results
print(f"🎯 Based on historical data, to consistently enroll {target_enrollment} students per cohort,")
print(f"you should aim to send out approximately {offers_needed:.0f} offers per cohort.")
print(f"📋 With a 23% no-show rate, you should extend around {waitlist_offers_needed:.0f} additional offers for the waitlist.")
print(f"💼 Total offers to send per cohort: {total_offers_needed:.0f}")



🎯 Based on historical data, to consistently enroll 11 students per cohort,
you should aim to send out approximately 19 offers per cohort.
📋 With a 20% no-show rate, you should extend around 3 additional offers for the waitlist.
💼 Total offers to send per cohort: 23


In [17]:

target_enrollment_per_cohort = 11

applications_submitted_per_cohort = 52

offers_extended_per_cohort = 19

cohorts_per_year = 6

no_show_rate = 0.23

# Calculate total offers needed per cohort, including waitlist offers
waitlist_offers_per_cohort = round(waitlist_offers_needed)

# Total offers per cohort (accepted + waitlist)
total_offers_per_cohort = offers_extended_per_cohort + waitlist_offers_per_cohort

# Calculate percentages
percentage_accepted_of_total_applicants = (target_enrollment_per_cohort / applications_submitted_per_cohort) * 100
percentage_offers_to_accepted = (offers_extended_per_cohort / total_offers_per_cohort) * 100
percentage_offers_to_waitlist = (waitlist_offers_per_cohort / total_offers_per_cohort) * 100

# Estimated total offers for the year
total_offers_per_year = total_offers_per_cohort * cohorts_per_year

# Output the calculated values
print(f"✅ Avg. accepted applicants per cohort: {target_enrollment_per_cohort}")
print(f"📆 Estimated annual acceptance capacity (6 cohorts): {target_enrollment_per_cohort * cohorts_per_year}")
print(f"🎯 To consistently enroll {target_enrollment_per_cohort} students per cohort,")
print(f"    you should aim for at least {applications_submitted_per_cohort} applications submitted per cohort.")
print(f"📊 Based on an average conversion rate of {(target_enrollment_per_cohort / applications_submitted_per_cohort) * 100:.2f}% from applications to enrolled.")
print(f"🎯 Based on historical data, to consistently enroll {target_enrollment_per_cohort} students per cohort,")
print(f"    you should aim to send out approximately {offers_extended_per_cohort} offers per cohort.")
print(f"📋 With a {no_show_rate * 100}% no-show rate, you should extend around {waitlist_offers_per_cohort} additional offers for the waitlist.")
print(f"💼 Total offers to send per cohort: {total_offers_per_cohort}")
print(f"Percentage of accepted applicants out of total applicants per cohort: {percentage_accepted_of_total_applicants:.2f}%")
print(f"Percentage of offers extended to accepted students: {percentage_offers_to_accepted:.2f}%")
print(f"Percentage of offers extended to waitlisted students: {percentage_offers_to_waitlist:.2f}%")


✅ Avg. accepted applicants per cohort: 11
📆 Estimated annual acceptance capacity (6 cohorts): 66
🎯 To consistently enroll 11 students per cohort,
    you should aim for at least 52 applications submitted per cohort.
📊 Based on an average conversion rate of 21.15% from applications to enrolled.
🎯 Based on historical data, to consistently enroll 11 students per cohort,
    you should aim to send out approximately 19 offers per cohort.
📋 With a 23.0% no-show rate, you should extend around 3 additional offers for the waitlist.
💼 Total offers to send per cohort: 22
Percentage of accepted applicants out of total applicants per cohort: 21.15%
Percentage of offers extended to accepted students: 86.36%
Percentage of offers extended to waitlisted students: 13.64%
